Generadores
===========

**Date:** 2023-11-07



## Ejemplo



In [1]:
import networkx as nx
g = nx.path_graph(5)
nx.draw(g, with_labels=True)

In [1]:
clanes = nx.find_cliques(g)
clanes

In [1]:
next(clanes)

Cuando un generador se termina, ejecutar `next` produce la excepción `StopIteration`. Podemos entonces enlistar los clanes de la siguiente manera:



In [1]:
clanes = nx.find_cliques(g)

while True:
    try:
        clan = next(clanes)
        print(clan)
    except StopIteration:
        break

## Problema



Supongamos que queremos determinar si una gráfica muy grande tiene al menos un clan de tamaño al menos 10. (Con 1000 vértices se puede esperar un dibujo, con 10000 vértices ya toma demasiado tiempo)



In [1]:
import networkx as nx
import matplotlib.pyplot as plt

fig, ax = plt.subplots(figsize=(20, 20))

g = nx.gnp_random_graph(1000, 0.05)
nx.draw(g, node_size=1, node_color='red')

In [1]:
clanes = nx.find_cliques(g)
clanes

In [1]:
next(clanes)

In [1]:
len(list(clanes))

In [1]:
clanes = nx.find_cliques(g)

b = 6

for q in clanes:
    if len(q) >= b:
        print(f"El clan {q} tiene al menos {b} vértices")

In [1]:
b = 7

clanes = nx.find_cliques(g)

for q in clanes:
    if len(q) >= b:
        print(f"El clan {q} tiene al menos {b} vértices")
        break # Si encuentra el clan, 'break' interrumpe el ciclo 'for'
else: # una cláusula 'else' en un ciclo 'for' se ejecuta si el ciclo se agotó
    print("No encontré tal clan.")

## Definir generadores



Un generador se puede definir usando la misma sintaxis que para una función, pero usando la palabra `yield` en lugar de `return`. Cada llamada a `next` ejecuta la función hasta encontrar el siguiente `yield`.



In [1]:
def saludos():
    yield "Hola"
    yield "Qué tal"
    yield "Buenos días"

salu = saludos()
salu

In [1]:
next(salu)

In [1]:
def pares(m, n):
    for i in range(m):
        for j in range(n):
            yield (i, j)

gener = pares(3, 4)
gener

In [1]:
next(gener)

In [1]:
gener2 = pares(3, 3)
list(gener2)

In [1]:
def cuadrados():
    i = 0
    while True:
        i = i+1
        yield i**2
    
cuads = cuadrados()
cuads

In [1]:
next(cuads)

In [1]:
def fibo_gen():
    i, j = 0, 1
    while True:
        yield i
        i, j = i+j, i

fibos = fibo_gen()
fibos

In [1]:
next(fibos)

También se puede obtener un generador por medio de una sintaxis parecida a la de las comprensiones de listas.



In [1]:
import networkx as nx

grafica = (nx.graph_atlas(i) for i in range(1253))
grafica

In [1]:
nx.draw(next(grafica))

## Leer archivos



Un archivo de texto se lee en Python por medio de un generador de sus líneas.



In [1]:
mis_datos = open("datos.csv")

In [1]:
next(mis_datos)

## Tarea



Hacer una función que, dada una lista de cadenas, informe cuántas veces ocurre cada cadena. Por ejemplo, si la lista es `['la', 'casa', 'roja', 'es', 'roja']`, debe reportar que 'la', 'casa' y 'es' aparecen una vez cada una y 'roja' aparece dos veces.

Guardar la letra de una canción en un archivo de texto. Hacer una función que reciba el nombre del archivo y reporte cuántas veces ocurre cada palabra.

**Sugerencias** Para la primer tarea, escoger una estructura de datos que sea adecuada para el problema. Para la segunda, usar el método `strip` de una cadena, la cual remueve el carácter de salto de línea del final de una línea.



## Generación de objetos



Primero, veremos que los generadores pueden también tomar los objetos que generan a partir de un *iterador* utilizando el comando `yield from`:



In [1]:
def beatles():
    yield "John"
    yield from ["Paul, George"]
    # yield from {"Paul":1, "George":0}
    # yield from ("Paul", "George")
    yield "Ringo"

list(beatles())

Esto nos puede servir para generar sucesiones de ceros y unos por medio de *backtracking*. La idea esencial del backtracking es construir un objeto (por ejemplo, un vector) poco a poco a través de sus componentes de manera recursiva.



In [1]:
def knapsack(l, X=()):
    if l == 4:
        yield X
    else:
        yield from knapsack(l+1, X+(1,))
        yield from knapsack(l+1, X+(0,))

gens = knapsack(0)        
list(gens)

Podemos usar esta idea para generar tuplas de ceros y unos para resolver el problema Knapsack.



In [1]:
my_profits = (3, 5, 8, 2, 7, 4, 5, 6)
my_weights = (1, 7, 5, 5, 6, 3, 2, 6)

def my_knapsack(weights, profits, capacity):
    def _generate(l, X=()):
        if l == len(weights):
            yield X
        else:
            yield from _generate(l+1, X + (1,))
            yield from _generate(l+1, X + (0,))
   
    optP = 0
    optX = ()
    n = len(weights)
    for curX in _generate(0):
        if sum([weights[i]*curX[i] for i in range(n)]) <= capacity:
            curP = sum([profits[i]*curX[i] for i in range(n)])
            if curP > optP:
                optP = curP
                optX = curX
    return optP, optX
            
my_knapsack(my_weights, my_profits, 17)

Sin embargo, esta función no sirve para poder aplicar el concepto de *podar* el árbol de búsqueda. En el siguiente código cada tupla se genera y se checa su factibilidad en una sola función.

Nótese que hemos tenido que introducir la instrucción `nonlocal` para que solo exista una copia de tales variables en cada recursión. Por un motivo parecido, hemos tenido que cargar con el parámetro `curX` en cada llamada recursiva.



In [1]:
my_profits = (3, 5, 8, 2, 7, 4, 5, 6, 1, 4, 6, 7, 10)
my_weights = (1, 7, 5, 5, 6, 3, 2, 6, 3, 3, 4, 3, 8)

def my_knapsack2(weights, profits, capacity):
    optP = 0
    optX = ()
    curP = 0
    n = len(weights)

    def _knapaux(l, curX):
        nonlocal optP, optX, curP
        if l == n:
            if sum([weights[i]*curX[i] for i in range(n)]) <= capacity:
                curP = sum([profits[i]*curX[i] for i in range(n)])
                if curP > optP:
                    optP = curP
                    optX = curX
        else:
            _knapaux(l+1, curX + (1,))
            _knapaux(l+1, curX + (0,))

    _knapaux(0, ())
    return optP, optX
            
my_knapsack2(my_weights, my_profits, 20)

Con esto, siguiendo el modelo del libro Kreher-Stinson, podemos mostrar el siguiente código que utiliza el concepto de podar el árbol de búsqueda (*pruning*).



In [1]:
my_profits = (3, 5, 8, 2, 7, 4, 5, 6, 1, 4, 6, 7, 10)
my_weights = (1, 7, 5, 5, 6, 3, 2, 6, 3, 3, 4, 3, 8)

def knapsack_pruning(weights, profits, capacity):
    optP = 0
    optX = ()
    curP = 0
    n = len(weights)
    Cl = []

    def _knapaux(l, curX, curW):
        nonlocal optP, optX, curP, Cl
        if l == n:
            if sum([profits[i]*curX[i] for i in range(n)]) > optP:
                optP = sum([profits[i]*curX[i] for i in range(n)])
                optX = curX
            Cl = []    
        else:
            if curW + weights[l] <= capacity:
                Cl = [1, 0]
            else:
                Cl = [0]
            for x in Cl:
                _knapaux(l+1, curX + (x,), curW + weights[l]*x)

    _knapaux(0, (), 0)
    return optP, optX
            
knapsack_pruning(my_weights, my_profits, 20)